In [1]:
from base_imputer import BaseImputer
from global_mean import GlobalMeanImputer
from random_copy import RandomTrainingSampleImputer
from metrics import METRICS

In [2]:
import pickle, os, json, math, itertools, numpy as np, pandas as pd
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from typing import Dict, List, Iterable, Tuple, Optional

## Evaluation functions

Every model in this hackathon will be tested with the same evaluation code.  
To make your model compatible, **stick to the API defined in `BaseImputer`** (see `base_imputer.py`) and demonstrated in the baselines (`random_copy.py`, `global_mean.py`).

Specifically, your model class must implement two functions:

1. **`fit(train, input_modalities, target_modalities)`**  
   - `train`: a dictionary of `{modality_name: DataFrame}` loaded from a pickle.  
     *Rows = samples, Columns = features*.  
   - `input_modalities`: a list of modality names that should be used as input.  
   - `target_modalities`: a list containing the **one modality** to be predicted.  

   This is where you initialize or train your model.

2. **`predict(eval_split, target_modalities)`**  
   - `eval_split`: another dictionary of `{modality_name: DataFrame}` (validation or test split).  
   - `target_modalities`: the same target modality list passed to `fit`.  

   This function should return a dictionary of `{target_modality: DataFrame}`,  
   where the DataFrame has the same index (samples) and columns (features) as the true target.

---

### Example

The `RandomTrainingSampleImputer` baseline shows the simplest valid implementation:

- `fit` just stores the training data.  
- `predict` copies rows from random training samples as predictions.

If your model follows the same pattern, it will work automatically with the shared `evaluate_model` function provided below.

---


In [3]:
def load_split_dict(path: str) -> Dict[str, pd.DataFrame]:
    with open(path, "rb") as f:
        d = pickle.load(f)
    assert isinstance(d, dict) and all(isinstance(v, pd.DataFrame) for v in d.values())
    return d

def evaluate_model(
    model: BaseImputer,
    train: Dict[str, pd.DataFrame],
    eval_split: Dict[str, pd.DataFrame],
    input_modalities: List[str],
    target_modality: str,
    metrics: Iterable[str] = ("mae", "rmse", "r2", "spearman", "pearson_featurewise","pearson_flat"),
):
    """
    Fit on train, predict on eval_split, and compute metrics for ONE target modality.
    """
    # Presence + alignment checks
    assert target_modality in train and target_modality in eval_split, \
        f"Missing modality '{target_modality}' in splits"
    assert (train[target_modality].columns == eval_split[target_modality].columns).all(), \
        f"Feature mismatch in '{target_modality}'"

    for m in input_modalities:
        assert m in train and m in eval_split, f"Missing input modality '{m}' in splits"

    model.fit(train, input_modalities, [target_modality])
    preds = model.predict(eval_split, [target_modality])

    y_true = eval_split[target_modality].to_numpy()
    y_pred = preds[target_modality].to_numpy()

    scores = {k: METRICS[k](y_true, y_pred) for k in metrics}

    results = {
        "model": model.name,
        "target": target_modality,
        "inputs": input_modalities,
        "metrics": scores,
    }
    return results, preds[target_modality]

## Example: Running a baseline model

Now that we have our train/val/test splits loaded, let’s see how to actually
**fit and evaluate** a model.

In this example:

- We pick one modality to be the **target** (the one we want to impute).  
- All the remaining modalities become the **inputs**.  
- We initialize the simple baseline `RandomTrainingSampleImputer`.  
- We call `evaluate_model`, which will:
  1. Fit the model on the training split,
  2. Use it to predict the target modality in the test split,
  3. Compute metrics (MAE, RMSE, R², Spearman, Pearson),
  4. Return both the results dictionary and the predicted DataFrame.

You can replace `RandomTrainingSampleImputer` with your own model class, and the
same workflow will work automatically.


In [10]:
train_path = "Data/ccle_split_train.pkl"
val_path   = "Data/ccle_split_val.pkl"
test_path  = "Data/ccle_split_test.pkl"

train_dict = load_split_dict(train_path)
val_dict   = load_split_dict(val_path)
test_dict  = load_split_dict(test_path)

sorted(train_dict.keys()), {m: df.shape for m, df in train_dict.items()}

(['metabolomics', 'methylation', 'miRNA', 'proteomics', 'rna'],
 {'metabolomics': (672, 225),
  'miRNA': (672, 734),
  'proteomics': (672, 214),
  'rna': (672, 2000),
  'methylation': (672, 2000)})

In [ ]:
target_modalities = list(train_dict.keys())[3]
input_modalities = [m for m in train_dict if m != target_modalities]
model = RandomTrainingSampleImputer(seed=42)
res,pre = evaluate_model(model, train_dict, test_dict, input_modalities, target_modalities)

In [14]:
res

{'model': 'random_copy',
 'target': 'rna',
 'inputs': ['metabolomics', 'miRNA', 'proteomics', 'methylation'],
 'metrics': {'mae': 0.8741846934134968,
  'rmse': 1.4268537213049715,
  'r2': -1.226296310726437,
  'spearman': -0.005542276685118547,
  'pearson_featurewise': -0.005639831756564695,
  'pearson_flat': -0.008441809328375242}}